

![JohnSnowLabs](https://nlp.johnsnowlabs.com/assets/images/logo.png)

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/JohnSnowLabs/spark-nlp-workshop/blob/master/tutorials/streamlit_notebooks/SENTIMENT_SW.ipynb)






# **Sentiment Analysis of Swahili texts**

## 1. Colab Setup

In [ ]:
!wget http://setup.johnsnowlabs.com/colab.sh -O - | bash

--2022-01-12 10:39:48--  http://setup.johnsnowlabs.com/colab.sh
Resolving setup.johnsnowlabs.com (setup.johnsnowlabs.com)... 51.158.130.125
Connecting to setup.johnsnowlabs.com (setup.johnsnowlabs.com)|51.158.130.125|:80... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://setup.johnsnowlabs.com/colab.sh [following]
--2022-01-12 10:39:48--  https://setup.johnsnowlabs.com/colab.sh
Connecting to setup.johnsnowlabs.com (setup.johnsnowlabs.com)|51.158.130.125|:443... connected.
HTTP request sent, awaiting response... 302 Moved Temporarily
Location: https://raw.githubusercontent.com/JohnSnowLabs/spark-nlp/master/scripts/colab_setup.sh [following]
--2022-01-12 10:39:49--  https://raw.githubusercontent.com/JohnSnowLabs/spark-nlp/master/scripts/colab_setup.sh
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:44

In [ ]:
!pip install --ignore-installed spark-nlp-display

     |████████████████████████████████| 95 kB 2.1 MB/s 
     |████████████████████████████████| 66 kB 3.6 MB/s 
     |████████████████████████████████| 15.7 MB 42.4 MB/s 
     |████████████████████████████████| 792 kB 45.8 MB/s 
     |████████████████████████████████| 11.3 MB 31.8 MB/s 
  Using cached spark_nlp-3.4.0-py2.py3-none-any.whl (140 kB)
     |████████████████████████████████| 1.1 MB 41.3 MB/s 
     |████████████████████████████████| 374 kB 46.1 MB/s 
     |████████████████████████████████| 102 kB 8.6 MB/s 
     |████████████████████████████████| 59 kB 5.0 MB/s 
     |████████████████████████████████| 1.6 MB 37.9 MB/s 
     |████████████████████████████████| 958 kB 42.8 MB/s 
     |████████████████████████████████| 100 kB 9.4 MB/s 
     |████████████████████████████████| 247 kB 31.8 MB/s 
     |████████████████████████████████| 503 kB 43.0 MB/s 
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the sour

In [ ]:
import pandas as pd
import numpy as np
import json
from pyspark.ml import Pipeline
from pyspark.sql import SparkSession
import pyspark.sql.functions as F
from sparknlp.annotator import *
from sparknlp.base import *
import sparknlp
from sparknlp.pretrained import PretrainedPipeline

## 2. Start Spark Session

In [ ]:
spark = sparknlp.start()

## 3. Some sample examples

In [91]:
text_list=["""Tukio bora katika sinema ilikuwa wakati Gerardo anajaribu kupata wimbo ambao unaendelea kupitia kichwa chake.""","""ni dharau kwa akili ya mtu na upotezaji mkubwa wa pesa""","""Kris Kristoffersen ni mzuri kwenye sinema hii na kweli hufanya tofauti.""","""Hadithi yenyewe ni ya kutabirika tu na ya uvivu.""","""ninapendekeza hizi kwa kuwa zinaonekana nzuri sana, kifahari na nzuri""","""Safaricom si muache kucheza na mkopo wa nambari yangu tafadhali. mnanifilisisha😓😓😯""","""Bidhaa ilikuwa bora na inafanya kazi vizuri kuliko ya verizon na bei ilikuwa rahisi ""","""Siwezi kuona jinsi sinema hii inavyoweza kuwa msukumo kwa mtu yeyote kushinda woga na kukataliwa.""","""Sinema hii inasawazishwa vizuri na vichekesho na mchezo wa kuigiza na nilijifurahisha sana."""]

## 4. Define Spark NLP pipeline

In [94]:
document_assembler = DocumentAssembler() \
      .setInputCol("text") \
      .setOutputCol("document")

tokenizer = Tokenizer() \
      .setInputCols(["document"]) \
      .setOutputCol("token")
    
normalizer = Normalizer() \
      .setInputCols(["token"]) \
      .setOutputCol("normalized")

stopwords_cleaner = StopWordsCleaner.pretrained("stopwords_sw", "sw") \
        .setInputCols(["normalized"]) \
        .setOutputCol("cleanTokens")\
        .setCaseSensitive(False)

embeddings = XlmRoBertaEmbeddings.pretrained("xlm_roberta_base_finetuned_swahili", "sw")\
    .setInputCols(["document", "cleanTokens"])\
    .setOutputCol("embeddings")

embeddingsSentence = SentenceEmbeddings() \
      .setInputCols(["document", "embeddings"]) \
      .setOutputCol("sentence_embeddings") \
      .setPoolingStrategy("AVERAGE")

sentimentClassifier = ClassifierDLModel.pretrained("classifierdl_xlm_roberta_sentiment", "sw") \
  .setInputCols(["document", "sentence_embeddings"]) \
  .setOutputCol("class")

sw_pipeline = Pipeline(stages=[document_assembler, tokenizer, normalizer, stopwords_cleaner, embeddings, embeddingsSentence, sentimentClassifier])

stopwords_sw download started this may take some time.
Approximate size to download 1.5 KB
[OK!]
xlm_roberta_base_finetuned_swahili download started this may take some time.
Approximate size to download 994.1 MB
[OK!]
classifierdl_xlm_roberta_sentiment download started this may take some time.
Approximate size to download 21.9 MB
[OK!]


## 5. Run the pipeline

In [95]:
model = sw_pipeline.fit(spark.createDataFrame(pd.DataFrame({'text': ['']})))

result = model.transform(spark.createDataFrame(pd.DataFrame({'text': text_list})))


## 6. Visualize results

In [96]:

result.select(F.explode(F.arrays_zip('document.result', 'class.result')).alias("cols")) \
.select(
        F.expr("cols['0']").alias("chunk"),
        F.expr("cols['1']").alias('result')).show(truncate=False)

+-------------------------------------------------------------------------------------------------------------+--------+
|chunk                                                                                                        |result  |
+-------------------------------------------------------------------------------------------------------------+--------+
|Tukio bora katika sinema ilikuwa wakati Gerardo anajaribu kupata wimbo ambao unaendelea kupitia kichwa chake.|Positive|
|ni dharau kwa akili ya mtu na upotezaji mkubwa wa pesa                                                       |Negative|
|Kris Kristoffersen ni mzuri kwenye sinema hii na kweli hufanya tofauti.                                      |Positive|
|Hadithi yenyewe ni ya kutabirika tu na ya uvivu.                                                             |Negative|
|ninapendekeza hizi kwa kuwa zinaonekana nzuri sana, kifahari na nzuri                                        |Positive|
|Safaricom si muache kucheza na 